In [47]:
from keras.models import Model
from keras.layers import Dense, Input
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np

In [48]:
iris = load_iris()
X = iris.data
y = iris.target

In [49]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)

In [50]:
inputs = Input(shape=(4,))
x = Dense(6, activation='relu')(inputs)
x = Dense(6, activation='relu')(x)
outputs = Dense(3, activation='softmax')(x)

In [51]:
model = Model(inputs=inputs, outputs=outputs)
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [52]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 4)                 0         
_________________________________________________________________
dense_10 (Dense)             (None, 6)                 30        
_________________________________________________________________
dense_11 (Dense)             (None, 6)                 42        
_________________________________________________________________
dense_12 (Dense)             (None, 3)                 21        
Total params: 93
Trainable params: 93
Non-trainable params: 0
_________________________________________________________________


In [53]:
model.fit(X_train, y_train, epochs=1000, verbose=0, batch_size=5)

In [54]:
pred = model.predict(X_test)

In [55]:
pred2 = np.argmax(pred, axis=1)

In [56]:
accuracy_score(pred2, y_test)

0.9666666666666667